## [Tutorial] Pima Indians Diabetes
### 책 <파이썬 머신러닝 완벽 가이드> 코드를 참조했습니다.

---

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
from sklearn.metrics import f1_score, confusion_matrix, precision_recall_curve, roc_curve
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

In [ ]:
data = pd.read_csv('../input/pima-indians-diabetes-database/diabetes.csv')
print(data['Outcome'].value_counts())
data.head(3)

**피마 인디언 당뇨병 feature information**

- `Pregnancies` : 임신 횟수

- `Glucose` : 포도당 부하 검사 수치

- `BloodPressure` : 혈압 (mm Hg)

- `SkinThickness` : 팔 삼두근 뒤쪽의 피하지방 측정값(mm)

- `Insulin` : 혈청 인슐린(mu U/ml)

- `BMI` : 체질량 지수

- `DiabetesPedigreeFunction` : 당뇨 내력 가중치 값

- `Age` : 나이

- `Outcome` : 클래스 결정 값(0 또는 1)

전체 768개의 데이터 중에서 negative 값 0이 500개, positive 값 1이 268로 상대적으로 negative가 많습니다.

In [ ]:
# feature type 확인
data.info()

In [ ]:
# 결측치 확인
data.isna().sum()

결측치는 없으며 피처 타입은 모두 숫자형입니다. 별도의 피처 인코딩은 필요없어 보입니다.

<br>
로지스틱 회귀를 위해 예측 모델을 생성합니다.

train_test_split을 이용하여 분리하고, 성능지표를 출력하여 재현율 곡선을 시각화 합니다.

In [ ]:
# train_test_split
X = data.iloc[:, :-1]
y = data.iloc[:, -1]

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=.2,
                                                    random_state=156,
                                                    stratify=y)

In [ ]:
# get_clf_eval function 생성
# confusion, accuracy, precision, recall, f1, roc_auc 나타냄

def get_clf_eval(y_test, pred=None, pred_proba=None):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    f1 = f1_score(y_test, pred)
    
    roc_auc = roc_auc_score(y_test, pred_proba)
    print('오차 행렬 confusion matrix')
    print(confusion)
    print('정확도 accuracy: {0:.4f}, 정밀도 precision: {1:.4f}, 재현율 recall: {2:.4f}, \F1: {3:.4f}, AUC: {4:.4f}'.format(accuracy, precision, recall, f1, roc_auc))

In [ ]:
# logistic regression
lr = LogisticRegression()
lr.fit(X_train, y_train)
pred = lr.predict(X_test)
pred_proba = lr.predict_proba(X_test)[:, 1]

get_clf_eval(y_test, pred, pred_proba)

예측 정확도가 약 77%, 재현율이 57%로 측정됩니다. 전체 데이터의 65%가 negative이기 때문에 정확도보다는 재현율에 조금 더 초점을 맞춰 봅니다.

먼저 정밀도 재현율 곡선을 보고 임곗값별 정밀도와 재현율 값의 변화를 확인하겠습니다.

In [ ]:
# precision_recall_curve_plot function
# 임계값 설정
thresholds = [.4, .45, .5, .55, .6]

def precision_recall_curve_plot(y_test, pred_proba_c1):
    # threshold ndarray와 이 threshold에 따른 정밀도, 재현율 ndarray 추출하기
    precisions, recalls, thresholds = precision_recall_curve(y_test, pred_proba_c1)
    
    # x축을 threshold값으로, y축은 정밀도, 재현율 값으로 각각 plot 수행. 정밀도는 점선으로 표시
    plt.figure(figsize=(8, 6))
    threshold_boundary = thresholds.shape[0]
    plt.plot(thresholds, precisions[0:threshold_boundary], linestyle='--', label='precision')
    plt.plot(thresholds, recalls[0:threshold_boundary], label='recall')
    
    # threshold 값 x축의 scale을 0, 1 단위로 변경
    start, end = plt.xlim()
    plt.xticks(np.round(np.arange(start, end, 0.1), 2))
    
    # x, y축 label과 legend, grid 설정
    plt.xlabel('Threshold value'); plt.ylabel('Precision and Recall value')
    plt.legend(); plt.grid()
    plt.show()

In [ ]:
pred_proba_c1 = lr.predict_proba(X_test)[:, 1]
precision_recall_curve_plot(y_test, pred_proba_c1)

임계값을 0.42 정도로 낮추면 정밀도와 재현율이 어느 정도 균형을 맞춥니다. 하지만 두 지표 모두 0.7이 안되는 수치로 보입니다.

임계값을 인위적으로 조작하기 전에 다시 데이터 값을 점검합니다.

In [ ]:
data.describe()

In [ ]:
# describe를 보면 `Glocose` 값의 min이 0으로 된 피처가 상당수 존재하는 것으로 보입니다. Glocuse는 포도당 수치이므로 0이 말이 안됩니다.
# 히스토그램으로 확인해보면 다음과 같습니다

plt.hist(data['Glucose'], bins=10)

In [ ]:
# min값이 0으로 되어 있는 피처에 대해 0 값의 건수와 전체 데이터 건수 대비 몇 퍼센트 비율로 존재하는 지 확인합니다
# 0값 체크할 feature list
zero_features = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']

# 전체 데이터 건수
total_count = data['Glucose'].count()

# feature 별로 반복하면서 데이터 값이 0인 데이터 건수와 퍼센트 계산
for feature in zero_features:
    zero_count = data[data[feature] == 0][feature].count()
    print('{0} 0 건수는 {1}, 퍼센트는 {2:.2f} %'.format(feature, zero_count,
                                                100 * (zero_count / total_count)))

In [ ]:
# 위 feature의 0값들을 평균값으로 대체합니다
mean_zero_features = data[zero_features].mean()
data[zero_features] = data[zero_features].replace(0, mean_zero_features)

In [ ]:
X = data.iloc[:, :-1]
y = data.iloc[:, -1]

# StandardScaler 클래스를 이용해 피처 데이터 세트에 일괄적으로 스케일링 적용
scaler = StandardScaler( )
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size = 0.2, random_state = 156, stratify=y)

# 로지스틱 회귀로 학습, 예측 및 평가 수행. 
lr_clf = LogisticRegression()
lr_clf.fit(X_train, y_train)
pred = lr_clf.predict(X_test)
pred_proba = lr_clf.predict_proba(X_test)[:, 1]

get_clf_eval(y_test, pred, pred_proba)

정밀도와 재현율이 일정 개선된 것을 확인할 수 있습니다.

<br>
추가적으로 재현율 수치 개선을 위해 임계값 변화 함수를 이용합니다.

In [ ]:
from sklearn.preprocessing import Binarizer

def get_eval_by_threshold(y_test , pred_proba_c1, thresholds):
    # thresholds 리스트 객체내의 값을 차례로 iteration하면서 Evaluation 수행.
    for custom_threshold in thresholds:
        binarizer = Binarizer(threshold=custom_threshold).fit(pred_proba_c1) 
        custom_predict = binarizer.transform(pred_proba_c1)
        print('임곗값:', custom_threshold)
        get_clf_eval(y_test , custom_predict, pred_proba_c1)

In [ ]:
thresholds = [.3, .33, .36, .39, .42, .45, .48, .5]
get_eval_by_threshold(y_test, pred_proba.reshape(-1, 1), thresholds)

output 정리하여 볼 때,

- 재현율을 높이는 데 가장 좋은 임계값은 0.33 (하지만 매우 극단적인 선택)

- 전체적인 성능 평가 지표 유지 + 재현율 조금 상승은 0.42


In [ ]:
# 앞선 로지스틱 회귀 모델을 이용해 임계값을 0.48로 낮춘 상태에서 다시 예측
binarizer = Binarizer(threshold = 0.48)

pred_th_048 = binarizer.fit_transform(pred_proba.reshape(-1, 1))
get_clf_eval(y_test, pred_th_048, pred_proba)